# Logica Proposizionale

## Riferimenti

In questa sezione esploriamo la logica proposizionale sfruttando il framework HOL di nholz. [Nholz](https://github.com/domasin/nholz) è semplicemente un porting in F# di [HOL Zero](http://www.proof-technologies.com/holzero/) che a sua volta è un dimostratore interattivo di teoremi sviluppato da Mark Adams in OCaml nello stile LCF della famiglia HOL.

Sfruttiamo il linguaggio definito da HolZero per esplorare la logica proposizionale con la guida dell'[Handbook of Practical Logic and Automated Reasoning](https://www.cl.cam.ac.uk/~jrh13/atp/) di John Harrison, seguendo passo passo il capitolo sulla logica proposizionale e riadattando le funzioni lì definite al linguaggio HOL del nostro framework. 

L'Handbook di John Harrison è accompagnato da codice sorgente in OCaml che è stato portato in F# da Eric Taucher, Jack Pappas, Anh-Dung Phan ed è disponibile su Github: [fsharp-logic-examples](https://github.com/jack-pappas/fsharp-logic-examples/). Nel ridefinire le funzioni dell'handbook riadattandole alla logica HOL si è utilizzata come riferimento proprio l'implementazione in F# del codice descritto nell'Handbook.

## Introduzione

La logica proposizionale studia espressioni che intendono rappresentare proposizioni, cioè  affermazioni che possono essere considerate vere o false e che chiameremo nel seguito semplicemente "formule". All'interno del framework HOL che utilizziamo, queste sono semplicemente termini di tipo `bool` che possono essere costruite da atomi booleani, costituiti dalle costanti `true` e `false` e da variabili di tipo `bool`, a cui sono applicati i connettivi logici proposizionali `~`, `/\`, `\/`, `<=>` e `<=>`. Le proposizioni atomiche variabili sono come le variabili nell'algebra ordinaria, e a volte ci riferiamo ad esse come variabili proposizionali o variabili booleane, le costanti (true e false), invece, possono essere viste come le costanti numeriche dell'algebra. Come suggerisce la parola "atomiche", non ne viene analizzata la struttura interna; questo porterebbe a considerare una logica predicativa che al momento non viene trattata. I connettivi proposizionali all'interno della logica HOL sono semplicemente funzioni da valori di verità a valori di verità.

## Avvio del motore logico

Per iniziare referenziamo il motore di nholz:

In [1]:
#r "nuget: nholz2"

open HOL


Installed Packages nholz2, 1.0.0

e istruiamo l'interprete .NET Interactive a restituire una rappresentazione concreta della sintassi dei tipi e dei termini piuttosto che la loro sintassi astratta interna al sistema:

In [2]:
module HolTypeAndTermsFormatter =
    Formatter.SetPreferredMimeTypesFor(typeof<hol_type> ,"text/html")
    Formatter.Register<hol_type>((fun ty -> print_type ty), "text/html")

    Formatter.SetPreferredMimeTypesFor(typeof<term> ,"text/html")
    Formatter.Register<term>((fun tm -> print_term tm), "text/html")
    Formatter.Register<term list>((fun xs -> 
                                    xs
                                    |> Seq.map (print_term)
                                    |> fun x -> sprintf "[%s]" (x |> String.concat ", ")),"text/html")
    Formatter.Register<term * term>((fun (x,y) -> 
        sprintf "(%s, %s)" (x |> print_term) (y |> print_term)
    ),"text/html")

in uno script F# inseriremmo invece:

    fsi.AddPrinter print_type
    fsi.AddPrinter print_term

Infine carichiamo almeno le teorie fino a `Bool` che contiene la definizione dei tipi e dei termini booleani e dei loro connettivi:

In [ ]:
CoreThry.load
Equal.load
Bool.load

Inoltre, per comodità, definiamo un'abbreviazione per la funzione `parse_term`:

In [14]:
let T = parse_term

## Operazioni sintattiche

Nell'handbook vengono definite una serie di funzioni corrispondenti ai costruttori dei vari tipi di formule proposizionali che svolgono la funzione inversa, cioè dividerle nelle loro componenti. Tali funzioni sono già presenti nel modulo [`Bool`](https://domasin.github.io/nholz/reference/hol-bool.html). Ad esempio:

In [15]:
(T "p /\ q")
|> dest_conj

(p:bool, q:bool)

Proviamo invece a ridefinire funzioni relative alla logica proposizionale che non sono già presenti nella libreria.

Intanto dobbiamo poter distinguere tra espressioni atomiche ed espressioni composte. A questo scopo definiamo `is_bool_atom` come una funzione che restituisce vero per termini booleani costanti o variabili.

In [18]:
/// the term is a boolean atom
let is_bool_atom tm = 
    tm |> is_bool_term && (tm |> is_const || tm |> is_var)

let pAndQ = "p /\ q" |> parse_term 
let pTerm = "p:bool" |> parse_term

printfn "%s is an atom? %b" (pAndQ |> print_term) (pAndQ |> is_bool_atom)
printfn "%s is an atom? %b" (pTerm |> print_term) (pTerm |> is_bool_atom)

p /\ q is an atom? false
p:bool is an atom? true


Sulle formule composte vogliamo eseguire delle operazioni sui loro atomi. A questo scopo definiamo `overatoms` che prende una funzione binaria `f`, un termine `tm` e un argomento `b` e applica `f` ad ogni atomo di `tm` come primo argomento e `b` come secondo argomento:

In [6]:
let rec overatoms f tm b =
    if tm |> is_bool_atom then 
        f tm b
    elif tm |> is_not then
        let p = tm |> dest_not
        overatoms f p b
    elif tm |> is_conj then
        let (p,q) = tm |> dest_conj
        overatoms f p (overatoms f q b)
    elif tm |> is_disj then
        let (p,q) = tm |> dest_disj
        overatoms f p (overatoms f q b)
    elif tm |> is_imp then
        let (p,q) = tm |> dest_imp
        overatoms f p (overatoms f q b)
    elif tm |> is_eq then
        let (p,q) = tm |> dest_eq
        overatoms f p (overatoms f q b)
    else failwith "check type annotation on eq"

Un'applicazione particolarmente comune è quella di raccogliere qualche insieme di attributi associati agli atomi; ritornando solamente, nel caso più semplice, l'insieme di tutti gli atomi. Possiamo far questo iterando una funzione f insieme con un "append" su tutti gli atomi, e convertendo infine il risultato in un insieme per rimuovere i duplicati. 

In [16]:
let atom_union f tm =
    (tm, [])
    ||> overatoms (fun h (t) -> (f h) @ t)
    |> setify

let atoms tm = 
    atom_union (fun a -> [a]) tm

(T @"p /\ q \/ r")
|> atoms

[p:bool, q:bool, r:bool]

## Semantica della logica proposizionale

Dal momento che le formule proposizionali intendono rappresentare asserzioni che possono essere vere o false, in ultima analisi il significato di una formula è semplicemente uno dei due valori di verità "vero" e "falso". Comunque, esattamente come un'espressione algebrica x + y + 1 ha un significato definito solo quando sappiamo per che cosa stanno le variabili x e y, il significato di una formula proposizionale dipende dai valori di verità assegnati alle sue formule atomiche. Questa assegnazione è codificata in una valutazione, che è una funzione dagli insiemi degli atomi all'insieme dei valori di verità {falso,vero}. Data una formula `p` e una valutazione `v` valutiamo il valore di verità complessivo con la seguente funzione definita ricorsivamente:

In [8]:
let rec eval v tm =
    if tm = false_tm then 
        false
    elif tm = true_tm then
        true
    elif tm |> is_bool_atom then 
        v tm
    elif tm |> is_not then 
        let p = tm |> dest_not
        not <| eval v p
    elif tm |> is_conj then 
        let (p,q) = tm |> dest_conj
        (eval v p) && (eval v q)
    elif tm |> is_disj then 
        let (p,q) = tm |> dest_disj
        (eval v p) || (eval v q)
    elif tm |> is_imp then 
        let (p,q) = tm |> dest_imp
        not(eval v p) || (eval v q)
    elif tm |> is_eq then 
        let (p,q) = tm |> dest_eq
        (eval v p) = (eval v q)
    else
        failwith "Not part of propositional logic."

Questa è la nostra definizione matematica della semantica della logica proposizionale, che intende costituire una formalizzazione delle nostre intuizioni. Ogni connettivo logico è interpretato da una corrispondente funzione boolean HOL. Per essere molto espliciti sul significato di questi operatori, possiamo elencare tutte le possibili combinazioni di input e vedere gli output corrispondenti.

Possiamo presentare questa informazione in una tavola di verità che mostri come il valore di verità di una formula è determinato dalle sue sotto formule immediate.

Così per i connettivi binari avremo:

$p$    | $q$   | $p \land q$  | $p \lor q$ | $p \Rightarrow q$| $p \Leftrightarrow q$|
-------|-------|-------|-------|-------|-------|
falso  | falso | falso | falso | falso | falso
falso  | vero  | falso | vero  | vero  | falso
vero   | falso | falso | vero  | falso | falso
vero   | vero  | vero  | vero  | vero  | vero

e per la negazione unaria:

$p$   | $\neg p$
------|---------
falso | vero
vero | falso

Proviamo a valutare una formula $p \land q \Rightarrow q \land r$ in una valutazione dove p, q e r sono impostati rispettivamente a "vero", "falso" e "vero". (Non ci preoccupiamo di definire il valore di atomi non coinvolti nella formula, e F# mostra un messaggio di warning che ci informa che non lo abbiamo fatto. Per evitarlo possiamo eventualmente sopprimere il warning avendo l'accortezza di reimpostarlo successivamente.)

In [9]:
(T "p /\ q ==> q /\ r")
|> eval (function 
    | tm when tm = (T "p:bool") -> true 
    | tm when tm = (T "q:bool") -> false 
    | tm when tm = (T "r:bool") -> true
)

True

In un'altra valutazione, comunque, la formula viene valutata a "falso":

In [10]:
(T "p /\ q ==> q /\ r")
|> eval (function 
    | tm when tm = (T "p:bool") -> true 
    | tm when tm = (T "q:bool") -> true 
    | tm when tm = (T "r:bool") -> false
)

False

## Tavole di verità meccanizzate

Intuitivamente sembra naturale che la valutazione di una formula sia indipendente dai valori assegnati dalla valutazione agli atomi che non occorrono nella formula (tralasciamo la dimostrazione che è riportata già nell'handbook). 

Poiché l'interpretazione di una formula proposizionale `p` dipende solo dall'azione della valutazione sull'insieme finito (diciamo di n elementi) `atoms(p)`, e può fare solo una di due scelte, il valore di verità finale è completamente determinato da tutte le $2^n$ scelte per questi atomi. Quindi possiamo estendere in modo naturale l'enumerazione dei valori degli atomi nella forma di una tavola di verità dalle operazioni base a formule arbitrarie. Per implementare questo in F#, iniziamo definendo una funzione che testa se una funzione `subfn` ritorna true su tutte le possibili valutazioni degli atomi `ats`, usando una valutazione esistente `v` per tutti gli altri atomi. Lo spazio di tutte le valutazioni è esplorato modificando successivamente `v` in modo da impostare ogni atomo `p` a "vero" e "falso" e richiamando ricorsivamente:

In [20]:
let rec onallvaluations subfn v ats =
    match ats with
    | [] -> subfn v
    | p :: ps ->
        let v' t q =
            if q = p then t
            else v q
        onallvaluations subfn (v' false) ps
        && onallvaluations subfn (v' true) ps

Possiamo applicare questa a una funzione che disegna una riga della tavola di verità e che ritorna "vero". (Il valore di ritorno è importante, perché `&&` valuterà il suo secondo argomento solo se il primo argomento è true.) Questo può quindi essere usato per disegnare l'intera tavola di verità per una formula:

In [26]:
let pname tm = 
    if tm |> is_const then 
        tm |> const_name
    elif tm |> is_var then 
        tm |> var_name
    else ""

let fprint_truthtable sw fm =
    // [P "p"; P "q"; P "r"]
    let ats = atoms fm
    // 5 + 1 = length of false + length of space
    let width = List.foldBack (max << String.length << pname) ats 5 + 1
    let fixw s = s + String.replicate (width - String.length s) " "
    let truthstring p = fixw (if p then "true" else "false")
    let mk_row v =
        let lis = ats |> map (fun x -> 
            match x with
            | Tmconst ("true", Tycomp ("bool", [])) -> fixw "true"
            | Tmconst ("false", Tycomp ("bool", [])) -> fixw "false"
            | _ -> truthstring(v x)
            )
        let ans = truthstring (eval v fm)
        fprintf sw "%s" (List.foldBack (+) lis ("| " + ans))
        fprintfn sw ""
        true
    let seperator = String.replicate (width * (List.length ats) + 9) "-"
    fprintfn sw "%s" (List.foldBack (fun s t -> fixw(pname s) + t) ats "| formula")
    fprintfn sw "%s" seperator
    let _ = onallvaluations mk_row (fun x -> false) ats
    fprintfn sw "%s" seperator
    fprintfn sw ""

let writeToString fn = 
    use sw = new System.IO.StringWriter()
    fn sw
    sw.ToString()

let inline print_truthtable f = fprint_truthtable stdout f
let inline sprint_truthtable f = writeToString (fun sw -> fprint_truthtable sw f)

Possiamo testare la nostra funzione di stampa su alcune formule:

In [29]:
(T "p /\ q ==> q /\ r")
|> print_truthtable

p     q     r     | formula
---------------------------
false false false | true  
false false true  | true  
false true  false | true  
false true  true  | true  
true  false false | true  
true  false true  | true  
true  true  false | false 
true  true  true  | true  
---------------------------



In [24]:
(T @"p /\ q \/ s ==> ~p \/ (r <=> s)")
|> print_truthtable

p     q     s     r     | formula
---------------------------------
false false false false | true  
false false false true  | true  
false false true  false | true  
false false true  true  | true  
false true  false false | true  
false true  false true  | true  
false true  true  false | true  
false true  true  true  | true  
true  false false false | true  
true  false false true  | true  
true  false true  false | false 
true  false true  true  | true  
true  true  false false | true  
true  true  false true  | false 
true  true  true  false | false 
true  true  true  true  | true  
---------------------------------



In [27]:
(T "true")
|> print_truthtable

true  | formula
---------------
true  | true  
true  | true  
---------------



In [28]:
(T @"(true ==> (x = false)) ==> ~(y \/ false /\ z)")
|> print_truthtable

true  x     false y     z     | formula
---------------------------------------
true  false false false false | true  
true  false false false true  | true  
true  false false true  false | false 
true  false false true  true  | false 
true  false false false false | true  
true  false false false true  | true  
true  false false true  false | false 
true  false false true  true  | false 
true  true  false false false | true  
true  true  false false true  | true  
true  true  false true  false | true  
true  true  false true  true  | true  
true  true  false false false | true  
true  true  false false true  | true  
true  true  false true  false | true  
true  true  false true  true  | true  
true  false false false false | true  
true  false false false true  | true  
true  false false true  false | false 
true  false false true  true  | false 
true  false false false false | true  
true  false false false true  | true  
true  false false true  false | false 
true  false false true 